<a href="https://colab.research.google.com/github/M1F1/MasterThesis/blob/master/PreprocessingIMDB_MR_toBTformat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import importlib
!pip install gdown==3.11.0
drive.mount('/content/gdrive')

if importlib.util.find_spec('lineflow') is None:
  !pip install lineflow
import lineflow as lf
import lineflow.datasets as lfds
import pandas as pd
import os
from pathlib import Path
from google.colab import drive
import numpy as np
from difflib import SequenceMatcher
!pip install python-Levenshtein
import Levenshtein
from tqdm import tqdm
from pathlib import Path
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

os.environ['DATASETS_PATH'] = str(Path()/'gdrive'/'My Drive'/'praca_magisterska'/'pytorch_lightning'/'datasets')





  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.11.0-cp36-none-any.whl size=9619 sha256=5007c6e75e91a92f907187f9e28264991fe6a99e50d9674299a55c9467c26f60
  Stored in directory: /root/.cache/pip/wheels/19/a6/67/ebb80360fc57bb0ddd5da77f57b275084cd8838bf7d5b91685
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4
  Created wheel for lineflow: filename=lineflow-0.6.4-cp36-none-any.whl size=23202 sha256=fad549caa0adb4276c6b309a7e8e4c51dd81b28218223de25bb37e18d08e5db8
  Stored in directory: /root/.cache/pip/wheels/b9/11/32/a6120f98d7d11ed8cf1b28b265a12a4b72842da341c13384c1
  Created wheel for arrayfiles: filename=arrayfiles-0.0.1-cp36-none-any.whl size=5434 sha256=e413826e6eeabe9fb5b4a9cb09b03cd7d396308d66c4a2a6b02d11c355c4ed85
  Stored in directory: /root/.cache/pip/wheels/2c/43/9b/f

### IMDB Dataset

In [ ]:
def preprocessor(text): 
  import re
  pattern1 = re.compile(r'<.*?>')
  # pattern2 = re.compile('[\W_]+')
  text = pattern1.sub('', text)
  # print('text after p1: ', text)
  # text = text.lower()
  return text

ds = lfds.Imdb('train') + lfds.Imdb('test')
ds = ds.map(lambda x: {'text': preprocessor(x[0]), 'intent': x[1]})
df = pd.DataFrame(ds)
df['bt'] = np.nan
df.to_csv(os.path.join(os.environ['DATASETS_PATH'], 'dumbed_imdb_oafa.csv'))

dfs = np.array_split(df, 50)
for i, df_chunk in enumerate(dfs): 
  path = os.path.join(os.environ['DATASETS_PATH'], f'imdb_file_{i}.txt')
  df_chunk['text'].to_csv(path, header=None, index=None, sep='\n', mode='w')

Reading from transcribed files


In [ ]:
global_review_list = []
for i in range(50):
  path_to_file = os.path.join(os.environ['DATASETS_PATH'], 'imdb_file_{}.txt'.format(i))
  with open(path_to_file, "r", encoding='utf-8') as bookFile:
      review_list = bookFile.readlines()
  global_review_list.extend(review_list)


global_review_list_bt = []
for i in range(50):
  path_to_file = os.path.join(os.environ['DATASETS_PATH'], 'imdb_file_bt_{}'.format(i), 'file_0_of_1.json')
  with open(path_to_file, "r", encoding='utf-8') as bookFile:
      review_list = bookFile.readlines()
  global_review_list_bt.extend(review_list)

df_bt =  pd.DataFrame({'text':global_review_list, 'bt': global_review_list_bt})

In [ ]:
import re, string

def delete_non_alphanumeric(text):
  return re.sub('[\W_]+', '', text)


In [ ]:

df_bt['text'] = df_bt['text'].replace(r'"','', regex=True)
df_bt['bt'] = df_bt['bt'].replace(r'"','', regex=True)
df['text'] = df['text'].replace(r'"','', regex=True)


df_bt['bt'] = df_bt['bt'].replace(r'\n','', regex=True)
df_bt['text'] = df_bt['text'].replace(r'\n','', regex=True)
df['text'] = df['text'].replace(r'\n','', regex=True)
# df_bt['text'] = df_bt['text'].apply(lambda x: [item for item in x if item not in stop])
# df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stop])

# df_bt['text'] = df_bt['text'].apply(delete_non_alphanumeric)
# df['text'] = df['text'].apply(delete_non_alphanumeric)

# df_bt['text'] = df_bt['text'].apply(lambda x: x[:200])
# df['text'] = df['text'].apply(lambda x: x[:200])

new_indexes_df = df.text.str.len().sort_values().index
new_indexes_df_bt = df_bt.text.str.len().sort_values().index

df = df.reindex(new_indexes_df)
df_bt = df_bt.reindex(new_indexes_df_bt)

df = df.reset_index(drop=True)
df_bt =  df_bt.reset_index(drop=True)

df['len']  = df['text'].str.len()
df_bt['len'] = df_bt['text'].str.len()

s = df['text'] == df_bt['text']
print(sum(s))
d = df['text'] != df_bt['text']




4777


In [ ]:

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio() 

for idx, rx in tqdm(df.iterrows(), total=df.shape[0]):

  df_temp = df_bt[df_bt['len'] > (rx['len'] - 1)]

  for idy, ry in df_temp.iterrows():

    ratio = similar(rx['text'], ry['text']) 

    if ratio > 0.97:
      df['bt'].iloc[idx] = df_bt['bt'].iloc[idy] 
      break


  0%|          | 0/50000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
100%|██████████| 50000/50000 [41:27<00:00, 20.10it/s]


In [ ]:
df.to_csv(os.path.join(os.environ['DATASETS_PATH'], "imdb_with_bt.csv"))

## MR dataset

In [ ]:
import toolz
if not Path('./txt_sentoken').exists():
  !wget https://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
  !tar -zxvf review_polarity.tar.gz

text = []
sentiment = []
directory_neg = './txt_sentoken/neg'
directory_pos = './txt_sentoken/pos'

def open_file_and_save_as_string(file_path: str) -> str:
  lines = []

  with open(file_path, "r", encoding='utf-8') as f:
      lines = f.readlines()
      
  flatten_lines = list(toolz.concat(lines))
  joined_str = "".join(flatten_lines)
  return joined_str 
    
def create_data_pair(directory: str, label_type: str) -> dict:
  texts = []
  labels = []
  pathlist = Path(directory).rglob('*.txt')
  for path in pathlist:
      path_in_str = str(path)
      texts.append(open_file_and_save_as_string(path_in_str))
      labels.append(label_type)

  return {'text': texts,
          'labels': labels}

data_neg = create_data_pair(directory_neg, 'neg')
df_neg = pd.DataFrame(data_neg)
data_pos = create_data_pair(directory_pos, 'pos')
df_pos = pd.DataFrame(data_pos)
df_all_data = pd.concat([df_neg, df_pos])
df_all_data = df_all_data.reset_index(drop=True)

df_all_data['text'] = df_all_data['text'].replace(r'\n','', regex=True)
df_all_data.to_csv(str(Path(os.environ['DATASETS_PATH']) / 'mr.csv'))
mr_df = pd.read_csv(str(Path(os.environ['DATASETS_PATH']) / 'mr.csv'), index_col=0)

##### Create txt format for bt

In [19]:
mr_df
path = os.path.join(os.environ['DATASETS_PATH'], f'mr_file.txt')
mr_df['text'].to_csv(path, header=None, index=None, sep='\n', mode='w')

##### Get back translated data

In [20]:

path_to_file = os.path.join(os.environ['DATASETS_PATH'], 'mr_file_bt', 'file_0_of_1.json')
with open(path_to_file, "r", encoding='utf-8') as bookFile:
    review_list = bookFile.readlines()

mr_df['paraphrases'] = review_list
mr_df['paraphrases'] = mr_df['paraphrases'].replace(r'\n','', regex=True)
path = os.path.join(os.environ['DATASETS_PATH'], 'mr_with_bt.csv')
mr_df.to_csv(path, header=None, index=None, sep='\n', mode='w')
  